In [3]:
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field 
from langchain.tools import StructuredTool 

class ArxivArgs(BaseModel):
    query:str = Field(description="Query de busca no ArXiv")
    
tool_arvix = StructuredTool.from_function(
	func=ArxivAPIWrapper(top_k_results=2).run,
	args_schema=ArxivArgs,
	name='arvix',
	description=(
		"Uma ferramenta em torno do Arvix.org."
		"Útil para quando você precisa responder a perguntas sobre Física, Matemática, "
		"Ciência da Compotutação,  Biologia Quantitativa, Finanças Quantitativas, Estatística, "
        "Engenharia Elétrica e Economia utilizando artigos científicos do arxiv.org. "
        "A entrada deve ser uma consulta de pesquisa em inglês."
	),
	return_direct=True
)


In [5]:
print(f"Descricao: {tool_arvix.description}")
print(f"Args: {tool_arvix.args}")


Descricao: Uma ferramenta em torno do Arvix.org.Útil para quando você precisa responder a perguntas sobre Física, Matemática, Ciência da Compotutação,  Biologia Quantitativa, Finanças Quantitativas, Estatística, Engenharia Elétrica e Economia utilizando artigos científicos do arxiv.org. A entrada deve ser uma consulta de pesquisa em inglês.
Args: {'query': {'title': 'Query', 'description': 'Query de busca no ArXiv', 'type': 'string'}}


In [7]:
tool_arvix.run({"query": "llm"})

"Published: 2024-06-13\nTitle: Large Language Models as Software Components: A Taxonomy for LLM-Integrated Applications\nAuthors: Irene Weber\nSummary: Large Language Models (LLMs) have become widely adopted recently. Research\nexplores their use both as autonomous agents and as tools for software\nengineering. LLM-integrated applications, on the other hand, are software\nsystems that leverage an LLM to perform tasks that would otherwise be\nimpossible or require significant coding effort. While LLM-integrated\napplication engineering is emerging as new discipline, its terminology,\nconcepts and methods need to be established. This study provides a taxonomy for\nLLM-integrated applications, offering a framework for analyzing and describing\nthese systems. It also demonstrates various ways to utilize LLMs in\napplications, as well as options for implementing such integrations.\n  Following established methods, we analyze a sample of recent LLM-integrated\napplications to identify releva

In [8]:
from langchain_experimental.tools.python.tool import PythonREPLTool
tool_repl = PythonREPLTool()
print(f"Descrição: {tool_repl.description}")
print(f"Args: {tool_repl.args}")

Descrição: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.
Args: {'query': {'title': 'Query', 'type': 'string'}}


In [9]:
tool_repl.run({"query":"print('oi')"})

Python REPL can execute arbitrary code. Use with caution.


'oi\n'

In [11]:
from langchain.agents import load_tools 

tools = load_tools(["stackexchange"])
tool_stack = tools[0]
print(f"Descrição: {tool_repl.description}")
print(f"Args: {tool_repl.args}")

Descrição: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.
Args: {'query': {'title': 'Query', 'type': 'string'}}


In [12]:
tool_stack.run({"query":"Langchain Agents"})

'Question: @roo_validator error when importing langchain.text_splitter Python\nI have been working through a <span class="highlight">LangChain</span> tutorial and have hit this problem when trying to import <span class="highlight">langchain</span> into python in vscode on macos 13.4.1. &hellip; /<span class="highlight">agents</span>/init.py&quot;, line 2, in \nfrom langchain.agents.agent import <span class="highlight">Agent</span>\nFile &quot;/Users/jamesallison/Desktop/Python Apps Temp/LangchainPDF/.venv/lib/python3.8/site-packages/<span class="highlight">langchain</span>/<span class="highlight">agents</span>/agent.py&quot;, line &hellip; \n\nQuestion: How to Stream Langchain (Create pandas dataframe agent) with Azurechatgpt results in Flask Api\nI want to stream my <span class="highlight">langchain</span> implementation as response in Flask api. The streaming is Taking place but while running the api in the postman, i can see the result only after the stream ends. &hellip; I am using

In [14]:
from langchain_community.agent_toolkits.file_management.toolkit import FileManagementToolkit

tool_kit = FileManagementToolkit(
	root_dir="files"
)

tools = tool_kit.get_tools()

for tool in tools:
    print(f"Name: {tool.name}")
    print(f"Descrição: {tool.description}")
    print(f"Args: {tool.args}\n")
    

Name: copy_file
Descrição: Create a copy of a file in a specified location
Args: {'source_path': {'title': 'Source Path', 'description': 'Path of the file to copy', 'type': 'string'}, 'destination_path': {'title': 'Destination Path', 'description': 'Path to save the copied file', 'type': 'string'}}

Name: file_delete
Descrição: Delete a file
Args: {'file_path': {'title': 'File Path', 'description': 'Path of the file to delete', 'type': 'string'}}

Name: file_search
Descrição: Recursively search for files in a subdirectory that match the regex pattern
Args: {'dir_path': {'title': 'Dir Path', 'description': 'Subdirectory to search in.', 'default': '.', 'type': 'string'}, 'pattern': {'title': 'Pattern', 'description': 'Unix shell regex, where * matches everything.', 'type': 'string'}}

Name: move_file
Descrição: Move or rename a file from one location to another
Args: {'source_path': {'title': 'Source Path', 'description': 'Path of the file to move', 'type': 'string'}, 'destination_path':

In [15]:
from langchain_community.agent_toolkits.file_management.toolkit import FileManagementToolkit

tool_kit = FileManagementToolkit(
	root_dir="arquivos",
	selected_tools=["write_file", "read_file", "file_search", "list_directory"]
)

tools = tool_kit.get_tools()

In [16]:
from langchain.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI 
from langchain_core.utils.function_calling import convert_to_openai_function

tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

prompt = ChatPromptTemplate.from_messages([
	("system", "Você é um assistente amigável chamada Isaac capaz de gerencia arquivos"),
	("user", "{input}")
])

chat = ChatOpenAI()

In [17]:
from langchain.agents.agent import AgentFinish

def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values['output']
    else:
        return tool_run[resultado.tool].run(resultado.tool_input)

In [18]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser() | roteamento

In [19]:
chain.invoke({"input": "O que você é capaz de fazer?"})

'Olá! Eu sou o assistente Isaac e posso te ajudar a gerenciar arquivos. Posso escrever texto em arquivos, ler o conteúdo de arquivos, buscar arquivos com base em padrões de regex e listar arquivos e diretórios em uma pasta específica. Se precisar de ajuda com alguma dessas tarefas, é só me chamar!'